## A description of the problem and a discussion of the background.

Mishelle's Company sent her on a job to Queens, New York and have reserved an accomodation for her at the LaGuardia Plaza, New York 11369. She is currently on a fittness challenge and would like to see it through. The problem is to locate a gym fairly close to Mishelle, and which has standard facilities.

## A description of how to solve the problem

First of all, we import the necessary libraries that we need to solve the solve the problem such as numpy, pamdas, follium, requests... We would be calling on foursquare APIs.
Next we use geocode to find the address the Hilton Hotels and Resorts on the map (i.e; its longlitude and latitude) after defining our foursquare credentials ofcourse. Then we search for nearby gyms, and recomend the gym with the most ratings to Mishelle, giving her its location.

##### Importing Libraries

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.55 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00  35.91 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  50.49 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.59 MB/s
vincent-0.4.4- 100% |###################

## Collecting the data that will be used to solve the problem

##### We define our foursquare credentials

In [2]:
CLIENT_ID = 'SKY2YBFEUQBP02SSF0VJKFSIBNVJLOAN142JND1AJ2RU0VFC' # your Foursquare ID
CLIENT_SECRET = 'OZB1WLGWPZNAODYSHSTYGPESUBCILV0HN00QZE4LRPYYK0D1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SKY2YBFEUQBP02SSF0VJKFSIBNVJLOAN142JND1AJ2RU0VFC
CLIENT_SECRET:OZB1WLGWPZNAODYSHSTYGPESUBCILV0HN00QZE4LRPYYK0D1


##### Getting the Latitude and Longitude values of the LaGuardia Plaza and  Resorts

In [3]:
address = 'LaGuardia Plaza, New York 11369'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7298918 -73.997401


##### passing a search query to foursquare for nearby gyms

In [4]:
search_query = 'Gyms'
radius = 2000
print(search_query + ' .... OK!')

Gyms .... OK!


##### Defining url

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SKY2YBFEUQBP02SSF0VJKFSIBNVJLOAN142JND1AJ2RU0VFC&client_secret=OZB1WLGWPZNAODYSHSTYGPESUBCILV0HN00QZE4LRPYYK0D1&ll=40.7298918,-73.997401&v=20180604&query=Gyms&radius=2000&limit=30'

##### Getting results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c0c7e8ddb04f5265e7f5f04'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/gaybar_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1d8941735',
      'name': 'Gay Bar',
      'pluralName': 'Gay Bars',
      'primary': True,
      'shortName': 'Gay Bar'}],
    'hasPerk': False,
    'id': '42829c80f964a52032221fe3',
    'location': {'address': '167 8th Ave',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStreet': 'btwn W 18th & W 19th St',
     'distance': 1439,
     'formattedAddress': ['167 8th Ave (btwn W 18th & W 19th St)',
      'New York, NY 10011',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.742593,
       'lng': -74.000588}],
     'lat': 40.742593,
     'lng': -74.000588,
     'postalCode': '10011',
     'state': 'NY'},
    'name': 'GYM Sportsbar',
    'referralId': 'v-1544322701'},
   {'catego

#####  Transfering relevant part of  JSON file into a pandas dataframe 

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,42829c80f964a52032221fe3,167 8th Ave,US,New York,United States,btwn W 18th & W 19th St,1439,"[167 8th Ave (btwn W 18th & W 19th St), New Yo...","[{'label': 'display', 'lat': 40.742593, 'lng':...",40.742593,-74.000588,10011,NY,GYM Sportsbar,v-1544322701
1,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,50fdd8e9e4b0c9eebd272ee9,101 Murray St,US,New York,United States,Greenwich,2094,"[101 Murray St (Greenwich), New York, NY 10007...","[{'label': 'display', 'lat': 40.71552359310045...",40.715524,-74.013421,10007,NY,Gym @ St. John's,v-1544322701
2,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,575b2829498e77357d734a04,43 Crosby St,US,New York,United States,NaN,911,"[43 Crosby St, New York, NY 10012, United States]","[{'label': 'display', 'lat': 40.72175745506477...",40.721757,-73.998684,10012,NY,Gotham Gym SoHo,v-1544322701
3,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,4c8f73d24e95236ac70084ca,101 Murray St,US,New York,United States,NaN,2077,"[101 Murray St, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.7151657896297,...",40.715166,-74.012526,10007,NY,Gym @ St. John's,v-1544322701
4,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,4fad804ee4b03d09569cb6b9,8 Spruce St,US,New York,United States,NaN,2388,"[8 Spruce St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.70911805588783...",40.709118,-74.004463,10038,NY,The Gym @ Spruce Street School,v-1544322701


In [8]:
dataframe.shape

(5, 17)

In [9]:
dataframe.columns

Index(['categories', 'hasPerk', 'id', 'location.address', 'location.cc',
       'location.city', 'location.country', 'location.crossStreet',
       'location.distance', 'location.formattedAddress',
       'location.labeledLatLngs', 'location.lat', 'location.lng',
       'location.postalCode', 'location.state', 'name', 'referralId'],
      dtype='object')

In [11]:
dataframe['address']=dataframe['location.address'] 
dataframe['lat']=dataframe['location.lat'] 
dataframe['lng']=dataframe['location.lng'] 
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,address,lat,lng
0,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,42829c80f964a52032221fe3,167 8th Ave,US,New York,United States,btwn W 18th & W 19th St,1439,"[167 8th Ave (btwn W 18th & W 19th St), New Yo...","[{'label': 'display', 'lat': 40.742593, 'lng':...",40.742593,-74.000588,10011,NY,GYM Sportsbar,v-1544322701,167 8th Ave,40.742593,-74.000588
1,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,50fdd8e9e4b0c9eebd272ee9,101 Murray St,US,New York,United States,Greenwich,2094,"[101 Murray St (Greenwich), New York, NY 10007...","[{'label': 'display', 'lat': 40.71552359310045...",40.715524,-74.013421,10007,NY,Gym @ St. John's,v-1544322701,101 Murray St,40.715524,-74.013421
2,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,575b2829498e77357d734a04,43 Crosby St,US,New York,United States,NaN,911,"[43 Crosby St, New York, NY 10012, United States]","[{'label': 'display', 'lat': 40.72175745506477...",40.721757,-73.998684,10012,NY,Gotham Gym SoHo,v-1544322701,43 Crosby St,40.721757,-73.998684
3,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,4c8f73d24e95236ac70084ca,101 Murray St,US,New York,United States,NaN,2077,"[101 Murray St, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.7151657896297,...",40.715166,-74.012526,10007,NY,Gym @ St. John's,v-1544322701,101 Murray St,40.715166,-74.012526
4,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,4fad804ee4b03d09569cb6b9,8 Spruce St,US,New York,United States,NaN,2388,"[8 Spruce St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.70911805588783...",40.709118,-74.004463,10038,NY,The Gym @ Spruce Street School,v-1544322701,8 Spruce St,40.709118,-74.004463


## Methodology

##### A map displaying the hotel and  the gyms

In [33]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='LaGuardia Plaza',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe.lat, dataframe.lng, dataframe.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Lets explore the Gotham gym SoHo which is most closest to the LaGuardia Plaza

In [34]:
venue_id = '575b2829498e77357d734a04'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/575b2829498e77357d734a04?client_id=SKY2YBFEUQBP02SSF0VJKFSIBNVJLOAN142JND1AJ2RU0VFC&client_secret=OZB1WLGWPZNAODYSHSTYGPESUBCILV0HN00QZE4LRPYYK0D1&v=20180604'

In [36]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['pageUpdates', 'specials', 'name', 'rating', 'beenHere', 'dislike', 'id', 'tips', 'canonicalUrl', 'createdAt', 'inbox', 'attributes', 'popular', 'likes', 'location', 'allowMenuUrlEdit', 'reasons', 'ok', 'contact', 'bestPhoto', 'photos', 'colors', 'categories', 'ratingColor', 'verified', 'stats', 'ratingSignals', 'timeZone', 'hereNow', 'listed', 'shortUrl'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': []},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1513382955,
  'height': 1920,
  'id': '5a34642bf1fdaf2316f9807a',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {'name': 'Swarm for iOS', 'url': 'https://www.swarmapp.com'},
  'suffix': '/17603030_rPpK3vCesvwJcHB9i2Q5BwvZwMCsXUstclPNfJffPZw.jpg',
  'visibility': 'public',
  'width': 1440},
 'canonicalUrl': 'https://foursquare.com/v/gotham-gym-soho/575b2829498e77357d734a04',
 'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
    'suffix': '.png'},
   'id': '52f2ab2ebcbc57f1066b8b47',
   'name': 'Boxing Gym',
   'pluralName': 'Boxing Gyms',
   'primary': True,
   'shortName': 'Boxing Gym'}],
 'colors': {'algoVersion': 3,
  'highlightColor': {'photoId': '5a34642bf1fdaf2316f9807a',
   'value': -14673896},
  'highlightTextColor': {'photoId': '5a34642

Lets now see ratings on the gym

In [37]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.3


This is a good rating. lets check the ratings of the other gyms to compare our options. 

In [38]:
venue_id = '42829c80f964a52032221fe3' # ID of GYM Sportsbar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.6


In [39]:
venue_id = '50fdd8e9e4b0c9eebd272ee9' # ID of Gym @ St. John's
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [40]:
venue_id = '4fad804ee4b03d09569cb6b9' # The Gym @ Spruce Street School
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [41]:
venue_id = '4c8f73d24e95236ac70084ca' # ID of Gym @ St. John's 2
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


From our results, we see that three of the five gyms within a 2km radius of the LaGuardian Plaza where Mishelle is have not been rated yet. However, the closest to the plaza is one of the two that have been rated and the good news is the Gotham gym SoHo which is most close to the LaGuardian Plaza has the higher rating


Lets now explore the tips of the Gotham gym SoHo

In [53]:
#result['response']['venue']['tips']['count']
venue_id = '575b2829498e77357d734a04'
limit = 15
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
results = requests.get(url).json()
tips = results['response']['tips']['items']
tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['agreeCount', 'disagreeCount', 'lang', 'id', 'authorInteractionType', 'user', 'type', 'canonicalUrl', 'text', 'todo', 'createdAt', 'logView', 'likes'])

In [57]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"New location in Soho is a bit temporary feeling, but once you get past the floorboards bouncing underneath your feet in the ring, the class is bound to kick your butt.",0,0,5816686238fa301a9687f070,Lucie,Pustova,female,48345869


In [62]:
venue_id = '42829c80f964a52032221fe3' # ID of GYM Sportsbar
limit = 15
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
results = requests.get(url).json()
tips2 = results['response']['tips']['items']
tip2 = results['response']['tips']['items'][0]
tip2.keys()

dict_keys(['agreeCount', 'type', 'disagreeCount', 'lang', 'lastVoteText', 'lastUpvoteTimestamp', 'photourl', 'user', 'photo', 'canonicalUrl', 'text', 'todo', 'createdAt', 'id', 'logView', 'likes'])

In [69]:
pd.set_option('display.max_colwidth', -1)

tips2_df = json_normalize(tips2) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips2_filtered = tips2_df.loc[:, filtered_columns]

# display tips
tips2_filtered

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Friendly gay sports bar with 2-4-1 happy hour until 9pm. If it’s open, go downstairs to the Locker Room for a pick-up game of Flip Cup.",4,0,4cff6c76506d6a31f1ab1e86,The Corcoran Group,NaN,none,95423


## Result

Of the 5 gym (Gym Sportsbar, Gym @ St. Johns, Gotham Gym Solo, Gym at St. Johns2 and The Gym @ Spruce Street School) identified by by foursquare to be within a 2km radius of the LaGuardian plaza, we have in an order of proximity to the LaGuardian: Gotham Gym Solo, Gym @ St. Johns, Gym at St. Johns2, The Gym @ Spruce Street School, Gym Sportsbar.
Only 2 of the five gyms have ratings. These are the Gym Sportsbar with arating of 7.6, and the Gotham Gym Soho with a higher rating of 8.3.
Each of these 2 gyms have only one tip from users of the gym. Seems like the Gotham Gym Soho is a new location but the class is bouund to "kick butt" according to Lucie Pustova. The tip on the Gotham Gym Soho has no agree or disagree count. The tip on the Gym Sportsbar opens up a new light that indicates that the Gym Sportsbar is a gay sports bar which is highly recomended by the way and has an agree count of 4 with no disagree count.

## Discussion

In [ ]:
Given or results above, a good recommendation to Mishelle would be the Gotham Gym Soho. It is closest to the 